### Imports

In [5]:
import numpy as np
import os
from datetime import datetime
import sys

import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten, MaxPooling2D, TimeDistributed
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
import sklearn as sk
from keras import regularizers
import platform


print(f"Python Platform: {platform.platform()}")
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Python Platform: macOS-13.3.1-arm64-arm-64bit
Tensor Flow Version: 2.9.0
Keras Version: 2.9.0

Python 3.10.8 | packaged by conda-forge | (main, Nov 22 2022, 08:25:29) [Clang 14.0.6 ]
Scikit-Learn 1.2.0
GPU is available


##  Load frames

#### Load new dataset standard

In [9]:
drowsines_levels = np.array(["Yawning", "NonYawning"])
DATA_PATH = os.path.join('./Yawning_Extracted_Values')
label_map = {label: num for num, label in enumerate(drowsines_levels)}

files, labels = [], []

for level in drowsines_levels:
    for video_index in range(1,90):
        window = []      
        for frame in range(1, 180):
            file_path = os.path.join(DATA_PATH, level, str(video_index).zfill(3), (str(frame) + ".npy"))
            if os.path.exists(file_path):
                res = np.load(file_path)        
                window.append(res)
            else:
                print(f"File at {file_path} doesn't exists")
        files.append(window)
        labels.append(label_map[level])
        print(f"File number: {video_index} processed for level {level}")

print(f"Processed! Current dataset shape: {np.array(files).shape}")

File number: 1 processed for level Yawning
File number: 2 processed for level Yawning
File number: 3 processed for level Yawning
File number: 4 processed for level Yawning
File number: 5 processed for level Yawning
File number: 6 processed for level Yawning
File number: 7 processed for level Yawning
File number: 8 processed for level Yawning
File number: 9 processed for level Yawning
File number: 10 processed for level Yawning
File number: 11 processed for level Yawning
File number: 12 processed for level Yawning
File number: 13 processed for level Yawning
File number: 14 processed for level Yawning
File number: 15 processed for level Yawning
File number: 16 processed for level Yawning
File number: 17 processed for level Yawning
File number: 18 processed for level Yawning
File number: 19 processed for level Yawning
File number: 20 processed for level Yawning
File number: 21 processed for level Yawning
File number: 22 processed for level Yawning
File number: 23 processed for level Yawni

#### Load new dataset fancy progress

In [ ]:
def processDataset():
    drowsines_levels = np.array(["Sleepy", "NonSleepy"])
    DATA_PATH = os.path.join('./Extracted_Values_Bigger_Confidence')
    label_map = {label: num for num, label in enumerate(drowsines_levels)}

    files, labels = [], []

    for level in drowsines_levels:
        for video_index in range(1,90):
            window = []      
            for frame in range(1, 1001):
                file_path = os.path.join(DATA_PATH, level, str(video_index), (str(frame) + ".npy"))
                if os.path.exists(file_path):
                    res = np.load(file_path)        
                    window.append(res)
                else:
                    print(f"File at {file_path} doesn't exists")
            files.append(window)
            labels.append(label_map[level])
            progress_percentage = (video_index / 89) * 100  # Calculate the percentage progress
            status = f"Progress: {progress_percentage:.2f}% | Level: {level}"
            print(status, end="\r")  # Overwrite the previous status

    print(f"Processed! Current dataset shape: {np.array(files).shape}")

In [4]:
np.array(files).shape

(178, 1000, 1434)

## Yawning Model training

## Prapare data

In [1]:
import cv2 as cv
import os
from os.path import exists
import numpy as np
import mediapipe as mp

objc[15345]: Class CaptureDelegate is implemented in both /Users/M374155/miniforge3/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x17ed465a0) and /Users/M374155/miniforge3/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x16a5ec860). One of the two will be used. Which one is undefined.
objc[15345]: Class CVWindow is implemented in both /Users/M374155/miniforge3/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x17ed465f0) and /Users/M374155/miniforge3/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x1175eca68). One of the two will be used. Which one is undefined.
objc[15345]: Class CVView is implemented in both /Users/M374155/miniforge3/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x17ed46618) and /Users/M374155/miniforge3/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x1175eca90). One of the two will be used. Which one is undefined.
objc[15345]: Class CVSlider is implemented in both /Users/M

In [2]:
def extractKeypoints(result):
    if result.multi_face_landmarks:
        for face_detected in result.multi_face_landmarks:
            face = np.array([[res.x, res.y, res.z] for res in face_detected.landmark]).flatten()
    else:
        face = np.zeros(1434)

    return face

In [3]:
def drawLandmarks(mp_face_mesh, results, frame): 
    mp_drawing = mp.solutions.drawing_utils  
    mp_drawing_styles = mp.solutions.drawing_styles
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            mp_drawing.draw_landmarks(image=frame, landmark_list=face_landmarks,
                                       connections=mp_face_mesh.FACEMESH_TESSELATION, 
                                       landmark_drawing_spec=None,
                                        connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())
            mp_drawing.draw_landmarks(image=frame, landmark_list=face_landmarks,
                                       connections=mp_face_mesh.FACEMESH_CONTOURS,
                                         landmark_drawing_spec=None,
                                           connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style())
            mp_drawing.draw_landmarks(image=frame, landmark_list=face_landmarks,
                                       connections=mp_face_mesh.FACEMESH_IRISES,
                                         landmark_drawing_spec=None,
                                           connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_iris_connections_style())

In [4]:
def revertColors(frame, mesh): 
    image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)  
    image.flags.writeable = False 
    results = mesh.process(image)
    image.flags.writeable = True
    image = cv.cvtColor(image, cv.COLOR_RGB2BGR)  
    
    return image, results 

In [5]:
DATA_PATH = os.path.join('./Yawning//')

for state in ["Yawning", "NonYawning"]:
    for number in range(1,91):
            videoIndex  = str(number).zfill(3)
            try:
                os.makedirs(os.path.join(DATA_PATH, state , str(videoIndex)))
            except:
                pass

In [ ]:
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils  
mp_drawing_styles = mp.solutions.drawing_styles

videoNumberRange = [1, 36, 2, 6, 8, 9, 12, 13, 15, 20, 23, 24, 31, 32, 33, 34, 35, 5]

saveVideoIndex = 1
for videoNumber in videoNumberRange:
    for state in ["glasses", "noglasses", "night_noglasses", "sunglasses", "nightglasses"]:
        videoIndex = str(videoNumber).zfill(3)
        videoPath = os.path.join('/Users/M374155/Desktop/New Dataset/Training_Evaluation_Dataset/Training Dataset/' +
                                 videoIndex + "/" + state + "/yawning.avi")
        cap = cv.VideoCapture(videoPath)
        textFileName = "_yawning_mouth.txt"
        textFilePath = os.path.join('/Users/M374155/Desktop/New Dataset/Training_Evaluation_Dataset/Training Dataset/' +
                                    videoIndex + "/" + state + "/" + videoIndex + textFileName)
        print("Processing video number: ", str(videoNumber), " for state ", state, " Save Index:  ", saveVideoIndex)

        with open(textFilePath, 'r') as file:
            yawningFrameCounter = 0
            nonYawningFrameCounter = 0
            for char in file.read():
                if char == "1":
                    savePath = os.path.join("./Yawning/Yawning/", str(saveVideoIndex).zfill(3), str(yawningFrameCounter))
                    yawningFrameCounter += 1
                else:
                    savePath = os.path.join("./Yawning/NonYawning/", str(saveVideoIndex).zfill(3), str(nonYawningFrameCounter))
                    nonYawningFrameCounter += 1

                with mp_face_mesh.FaceMesh(
                                refine_landmarks=True,
                                min_detection_confidence=0.6,
                                min_tracking_confidence=0.7) as mesh:
                    
                    ret, frame = cap.read()               
                    image, results = revertColors(frame, mesh)
                    drawLandmarks(mp_face_mesh, results, image)
                    landmarks = extractKeypoints(results)

                    np.save(savePath, landmarks)
                    cv.imshow('Video', image)
                                    
                    if cv.waitKey(1) == ord('q'):
                        break              

        cap.release()
        saveVideoIndex += 1
cv.destroyAllWindows()


## Extract eyes values

In [8]:
DATA_PATH = os.path.join('./Eyes_Extracted_Values//')

for state in ["Sleepy", "NonSleepy"]:
    for number in range(1,91):
            videoIndex  = str(number).zfill(3)
            try:
                os.makedirs(os.path.join(DATA_PATH, state , str(videoIndex)))
            except:
                pass

: 

In [ ]:
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils  
mp_drawing_styles = mp.solutions.drawing_styles

videoNumberRange = [1, 36, 2, 6, 8, 9, 12, 13, 15, 20, 23, 24, 31, 32, 33, 34, 35, 5]

saveVideoIndex = 1
for videoNumber in videoNumberRange:
    for state in ["glasses", "noglasses", "night_noglasses", "sunglasses", "nightglasses"]:
        videoIndex = str(videoNumber).zfill(3)
        videoPath = os.path.join('/Users/M374155/Desktop/New Dataset/Training_Evaluation_Dataset/Training Dataset/' +
                                 videoIndex + "/" + state + "/slowBlinkWithNodding.avi")
        cap = cv.VideoCapture(videoPath)
        textFileName = "_slowBlinkWithNodding_eye.txt"
        textFilePath = os.path.join('/Users/M374155/Desktop/New Dataset/Training_Evaluation_Dataset/Training Dataset/' +
                                    videoIndex + "/" + state + "/" + videoIndex + textFileName)
        print("Processing video number: ", str(videoNumber), " for state ", state, " Save Index:  ", saveVideoIndex)

        with open(textFilePath, 'r') as file:
            sleepyEyesFrameCounter = 0
            nonSleepyEyesFrameCounter = 0
            for char in file.read():
                if char == "1":
                    savePath = os.path.join("./Eyes_Extracted_Values/Sleepy/", str(saveVideoIndex).zfill(3), str(sleepyEyesFrameCounter))
                    sleepyEyesFrameCounter += 1
                else:
                    savePath = os.path.join("./Eyes_Extracted_Values/NonSleepy/", str(saveVideoIndex).zfill(3), str(nonSleepyEyesFrameCounter))
                    nonSleepyEyesFrameCounter += 1

                with mp_face_mesh.FaceMesh(
                                refine_landmarks=True,
                                min_detection_confidence=0.6,
                                min_tracking_confidence=0.7) as mesh:
                    
                    ret, frame = cap.read()               
                    image, results = revertColors(frame, mesh)
                    drawLandmarks(mp_face_mesh, results, image)
                    landmarks = extractKeypoints(results)

                    np.save(savePath, landmarks)
                    cv.imshow('Video', image)
                                    
                    if cv.waitKey(1) == ord('q'):
                        break              

        cap.release()
        saveVideoIndex += 1
cv.destroyAllWindows()

# Define model

In [12]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='tanh', input_shape=(1000, 1434)))
model.add(LSTM(128, return_sequences=True, activation='tanh')) 
model.add(LSTM(32, return_sequences=False, activation='tanh')) 
model.add(Dense(64, activation='softmax', kernel_regularizer='l2')) 
model.add(Dropout(0.1))
model.add(Dense(drowsines_levels.shape[0], activation='softmax'))

model.summary()
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_72 (LSTM)              (None, 1000, 64)          383744    
                                                                 
 lstm_73 (LSTM)              (None, 1000, 128)         98816     
                                                                 
 lstm_74 (LSTM)              (None, 32)                20608     
                                                                 
 dense_24 (Dense)            (None, 64)                2112      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_25 (Dense)            (None, 2)                 130       
                                                                 
Total params: 505,410
Trainable params: 505,410
Non-t

#### Train Test Split Validation

In [ ]:
now = datetime.now()
date_string = now.strftime("%d/%m/%Y %H:%M:%S")

X = np.array(files)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

log_dir = os.path.join(f'Logs/BatchSizesTuning/Logs_train_test_split_batch_100')
tb_callback = TensorBoard(log_dir=log_dir)
es = EarlyStopping(monitor='val_loss', patience=50)

# model = Sequential()
# model.add(LSTM(64, return_sequences=True, activation='tanh', input_shape=(1099, 1434)))
# model.add(LSTM(128, return_sequences=True, activation='tanh')) 
# model.add(LSTM(32, return_sequences=False, activation='tanh')) 
# model.add(Dense(32, activation='softmax', kernel_regularizer='l2')) 
# model.add(Dense(drowsines_levels.shape[0], activation='softmax'))

# model.summary()
# model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])


model.fit(X_train, y_train, epochs=500, batch_size=100, shuffle=True, callbacks=[tb_callback, es], validation_split = 0.2) 

model.save(f'Models/drowsines_levels_train_test_split_${date_string}.h5')

### Cross validation

In [ ]:
from sklearn.model_selection import KFold

now = datetime.now()
date_string = now.strftime("%d/%m/%Y %H:%M:%S")

X = np.array(files)
y = to_categorical(labels).astype(int)

es = EarlyStopping(monitor='val_loss', patience=50)

log_dir = os.path.join(f'Logs/Logs_{date_string}_CrossValid')
tb_callback = TensorBoard(log_dir=log_dir)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = []
for train, test in kfold.split(X):
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, activation='tanh', input_shape=(1100, 1434)))
    model.add(LSTM(128, return_sequences=True, activation='tanh')) 
    model.add(LSTM(32, return_sequences=False, activation='tanh')) 
    model.add(Dense(64, activation='softmax', kernel_regularizer='l2')) 
    model.add(Dropout(0.1))
    model.add(Dense(drowsines_levels.shape[0], activation='softmax'))

    model.summary()
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(X[train], y[train], epochs=1000, batch_size=250, shuffle=True, callbacks=[tb_callback, es], validation_split=0.2) 

    scores = model.evaluate(X[test], y[test], verbose=0)
    print(f'Accuracy: {scores[1]*100}%')
    cv_scores.append(scores[1])

print(f'Cross-validation accuracy: {np.mean(cv_scores)*100:.2f}% +/- {np.std(cv_scores)*100:.2f}%')
model.save(f'Models/drowsines_levels_cross_validation_${date_string}.h5')

#### Find hyperparameters


In [ ]:
from sklearn.model_selection import KFold

now = datetime.now()
date_string = now.strftime("%d/%m/%Y %H:%M:%S")

X = np.array(files)
y = to_categorical(labels).astype(int)

es = EarlyStopping(monitor='val_loss', patience=50)

log_dir = os.path.join(f'Logs/BatchSizesTuning/Logs_{date_string}_CrossValid')
tb_callback = TensorBoard(log_dir=log_dir)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

batch_sizes = [50, 100, 150, 200]
batch_size_scores = []
cv_scores = []

for batch_size in batch_sizes:

    for train, test in kfold.split(X):
        model = Sequential()
        model.add(LSTM(64, return_sequences=True, activation='tanh', input_shape=(1000, 1434)))
        model.add(LSTM(128, return_sequences=True, activation='tanh')) 
        model.add(LSTM(32, return_sequences=False, activation='tanh')) 
        model.add(Dense(32, activation='softmax', kernel_regularizer='l2')) 
        model.add(Dense(drowsines_levels.shape[0], activation='softmax'))

        model.summary()
        model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

        model.fit(X[train], y[train], epochs=1000, batch_size=batch_size, shuffle=True, callbacks=[tb_callback, es], validation_split=0.2) 

        scores = model.evaluate(X[test], y[test], verbose=0)
        print(f'Accuracy: {scores[1]*100}%')
        cv_scores.append(scores[1])

    cross_validation_accuracy = np.mean(cv_scores)*100
    cross_validation_accuracy_error = np.std(cv_scores)*100
    print(f'Cross-validation accuracy: {cross_validation_accuracy:.2f}% +/- {cross_validation_accuracy_error:.2f}% for batch size: {batch_size}')
    batch_size_scores.append(dict(accuracy = cross_validation_accuracy, accuracy_error = cross_validation_accuracy_error, batch_size_used = batch_size))
    
    # model.save(f'Models/drowsines_levels_cross_validation_${date_string}.h5')

In [ ]:
for score in batch_size_scores:
    print(score)

#### Grid search for parameters

### Other architecture experiments

In [ ]:
now = datetime.now()
date_string = now.strftime("%d/%m/%Y %H:%M:%S")

X = np.array(files)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# es = EarlyStopping(monitor='val_loss', patience=50)

log_dir = os.path.join(f'Logs_15')
tb_callback = TensorBoard(log_dir=log_dir)

model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='tanh', input_shape=(1000, 1434)))
model.add(LSTM(128, return_sequences=True, activation='tanh')) 
model.add(TimeDistributed(Dropout(0.1)))
model.add(LSTM(32, return_sequences=False, activation='tanh')) 
model.add(Dense(32, activation='softmax', kernel_regularizer=regularizers.l2(0.008))) 
model.add(Dense(drowsines_levels.shape[0], activation='softmax'))

model.summary()
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=500, batch_size=50, shuffle = True, callbacks=[tb_callback], validation_split = 0.2) 

model.save(f'drowsines_weights_noStop_l2008_drop01_${date_string}.h5')

## Model validation

### Confusion Matrix

In [5]:
def generateConfusionMatrix():
    yhat = model.predict(X_test)
    ytrue = np.argmax(y_test, axis=1).tolist()
    yhat = np.argmax(yhat, axis=1).tolist()
    multilabel_confusion_matrix(ytrue, yhat)


NameError: name 'model' is not defined

### Tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir Logs/Logs_15/04/2023 21:41:01

## Links

In [ ]:
https://machinelearningmastery.com/use-dropout-lstm-networks-time-series-forecasting/ - LSTM Dropout
https://openai.com/research/how-ai-training-scales - OpenAI odnosnie bathc size
https://towardsdatascience.com/simple-guide-to-hyperparameter-tuning-in-neural-networks-3fe03dad8594 - hyperparameters